In [95]:
import os
import numpy as np
from ete3 import Tree

class TreeEncoder:

    def encode_tree(self, tree_str):
        """
        Encode the tree structure into a format suitable for input into the neural network.
        """
        # Check if the tree is already encoded
        csv_file = tree_str[:-4] + '.csv'
        if os.path.exists(csv_file):
            return csv_file
        # Call the external script to get CDV encoding
        cmd = f"python -m CDV_full_tree -t {tree_str} > {tree_str[:-4]}.csv"
        os.system(cmd)

    def encode_all_trees(self, trees_directory):
        """
        Encode all the trees in the given directory.
        """
        tree_files = [os.path.join(trees_directory, file) for file in os.listdir(trees_directory) if file.endswith('.nwk')]
        for tree_file in tree_files:
            print(tree_file)
            self.encode_tree(tree_file)


In [ ]:
trees_directory = "trees/musse/"
encoder = TreeEncoder()
encoded_trees = encoder.encode_all_trees(trees_directory)

In [141]:
import pandas as pd

cutoff = 100  # Number of trees to process

# Load parameter values as a dataframe
tree_files = [os.path.join(trees_directory, file) for file in os.listdir(trees_directory) if file.endswith('.nwk')]
parameter_files = [os.path.join(trees_directory, file[:-4] + '.params') for file in os.listdir(trees_directory) if file.endswith('.nwk')]

def process_params_musse(param_file):
    with open(param_file, 'r') as f:
        params = f.readlines()
        num_states = int(params[0].split()[1])
        lambdas = [float(row.split()[1]) for row in params[1:1 + num_states]]
        mus = [float(row.split()[1]) for row in params[1 + num_states:1 + 2 * num_states]]
        qs = [float(row.split()[1]) for row in params[1 + 2 * num_states:]]
        return lambdas, mus, qs

# Create a dataframe to store the parameter values
column_names = ['lambda1', 'lambda2', 'lambda3', 'mu1', 'mu2', 'mu3', 'q12', 'q13', 'q21', 'q23', 'q31', 'q32']
param_df = pd.DataFrame(columns=column_names)
for param_file in parameter_files:
    lambdas, mus, qs = process_params_musse(param_file)
    new_row = pd.DataFrame([lambdas + mus + qs], columns=column_names)
    
    param_df = pd.concat([param_df, pd.DataFrame([lambdas + mus + qs], columns=column_names)], ignore_index=True)   

# loading tree encodings/representations
# encoding has the following structure: 1 value of tree height, 500 values for tip states ('1' or '2')
# 1 value for tree height and 500 values for internal node heights
# + 2 values for nb of tips of each type (to be removed) and 1 value of rescaling (removed, but stocked for rescaling predicted values back to the original scale)

encoded_tree_files = [os.path.join(trees_directory, file) for file in os.listdir(trees_directory) if file.endswith('.csv')]
encoded_trees = pd.DataFrame()
for encoded_tree_file in encoded_tree_files:
    encoded_tree = pd.read_csv(encoded_tree_file, sep="\t", header=None, index_col=0, skiprows=1)  # Skip the header row
    encoded_trees = pd.concat([encoded_trees, encoded_tree], ignore_index=True)

# make sure there is correspondance between indexes of dataframe with parameter values and encodings
print(param_df.shape)
print(encoded_trees.shape)
param_df.index = encoded_trees.index
print(encoded_trees.head())

# split




C:\Users\matti\AppData\Local\Temp\ipykernel_2284\2448671926.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  param_df = pd.concat([param_df, pd.DataFrame([lambdas + mus + qs], columns=column_names)], ignore_index=True)


(245, 12)
(245, 502)
        1         2         3         4         5         6         7    \
0  6.866735  5.767957  4.863890  3.943322  6.208207  6.504332  6.854792   
1  4.675823  3.496983  3.219939  4.381005  4.644829  2.699579  2.623678   
2  6.650845  4.359031  6.405473  6.275414  5.912631  4.167767  2.134146   
3  8.588370  8.091624  7.742888  7.574931  7.423699  5.879166  7.452799   
4  7.745109  6.936440  6.698193  5.980017  7.499169  7.624462  5.845326   

        8         9         10   ...  493  494  495  496  497  498  499  500  \
0  3.583706  5.303253  6.417531  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1  2.427730  3.084728  2.946469  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  2.236878  5.775362  0.644143  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
3  7.264826  8.453792  4.531270  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4  3.787765  5.480250  3.545348  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

   501       502  
0  0.0  1.309473  
1  0.0  1

In [72]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from ete3 import Tree

class TreeEncoder:
    
    def __init__(self, max_length=512):
        self.max_length = max_length
        self.node_feature_size = 32  # Number of features to encode for each node in the tree

    def parse_newick(self, newick_str):
        """
        Parse Newick tree string using ete3, handling node labels.
        """
        return Tree(newick_str, format=1)

    def traverse_tree(self, tree, node=None):
        """
        Recursively traverse the tree to gather information about nodes and edges.
        """
        if node is None:
            node = tree.get_tree_root()

        if node.is_leaf():
            return [node.name]
        else:
            children = [self.traverse_tree(tree, child) for child in node.children]
            return children

    def flatten_tree_info(self, tree_info):
        """
        Flatten the nested tree structure info into a 1D array.
        """
        flattened = []
        for node_info in tree_info:
            if isinstance(node_info, list):
                flattened.extend(self.flatten_tree_info(node_info))
            else:
                flattened.append(node_info)
        return flattened

    def encode_tree(self, tree_str):
        """
        Encode the tree structure into a format suitable for input into the neural network.
        """
        # Call the external script to get CDV encoding
        cmd = f"CDV_full_tree.py -t {tree_str}"
        os.system(cmd)
        
        # Now read the generated CSV file
        csv_file = f"{tree_str[:-4]}.csv"
        encoded_tree = np.loadtxt(csv_file, delimiter=",")
        
        # Pad or truncate to match max_length
        if len(encoded_tree) < self.max_length:
            padding = np.zeros((self.max_length - len(encoded_tree), encoded_tree.shape[1]))
            encoded_tree = np.concatenate((encoded_tree, padding), axis=0)
        elif len(encoded_tree) > self.max_length:
            encoded_tree = encoded_tree[:self.max_length, :]
        
        return encoded_tree


def load_data(trees_directory):
    """
    Load tree and parameter files from the specified directory.
    """
    tree_files = [os.path.join(trees_directory, file) for file in os.listdir(trees_directory) if file.endswith('.nwk')]
    parameter_files = [os.path.join(trees_directory, file[:-4] + '.params') for file in os.listdir(trees_directory) if file.endswith('.nwk')]
    return tree_files, parameter_files

def process_data(tree_files, parameter_files):
    """
    Process tree and parameter data.
    """
    encoder = TreeEncoder()
    X, y = [], []
    for tree_file, param_file in zip(tree_files, parameter_files):
        with open(tree_file, 'r') as f:
            tree_str = f.read().strip()
            x = encoder.encode_tree(tree_str)
            print("Encoded tree shape:", x.shape)  # Print the shape of the encoded tree for debugging
            X.append(x)
        
        with open(param_file, 'r') as f:
            params = f.readlines()
            num_states = int(params[0].split()[1])
            lambdas = [float(row.split()[1]) for row in params[1:1 + num_states]]
            mus = [float(row.split()[1]) for row in params[1 + num_states:1 + 2 * num_states]]
            qs = [float(row.split()[1]) for row in params[1 + 2 * num_states:]]

            y.append(np.concatenate([lambdas, mus, qs]))

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)
    y = y / y.sum(dim=-1, keepdim=True)  # Normalize y values

    return X, y

# Load data
trees_directory = "trees/musse"
tree_files, parameter_files = load_data(trees_directory)

# Process data
X, y = process_data(tree_files, parameter_files)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


FileNotFoundError: ((sp27:1.250340508,sp28:1.250340508)nd15:7.74146153,((((sp30:0.6971156603,(sp44:0.05448322787,sp45:0.05448322787)nd43:0.6426324324)nd39:0.6993506711,sp26:1.396466331)nd17:4.57236873,(((((sp39:0.2613079878,sp40:0.2613079878)nd40:0.7983119657,(sp31:0.6836493874,sp32:0.6836493874)nd41:0.3759705661)nd36:2.581591996,(sp21:2.047336495,(sp33:0.5882194838,sp34:0.5882194838)nd38:1.459117012)nd30:1.593875454)nd23:0.2542306771,((sp41:0.2506839765,(sp42:0.2329470068,sp43:0.2329470068)nd46:0.01773696969)nd44:0.2999376771,sp35:0.5506216536)nd24:3.344820973)nd22:0.4035940053,((sp16:2.622670915,(sp24:1.438818684,sp25:1.438818684)nd37:1.183852231)nd25:1.205457832,(((sp46:0.01563795384,sp47:0.01563795384)nd45:0.458918388,sp38:0.4745563419)nd42:0.3877672302,sp29:0.862323572)nd32:2.965805175)nd21:0.470907884)nd13:1.66979843)nd10:1.06711515,((((sp22:1.711465293,sp23:1.711465293)nd34:1.935683526,(sp36:0.5114053313,sp37:0.5114053313)nd33:3.135743488)nd19:0.7528428002,sp10:4.39999162)nd14:1

## BiSSE

In [28]:
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from ete3 import Tree

# Load trees and corresponding parameters from the specified directory
def load_data_from_directory(directory):
    """
    Load trees and corresponding parameters from the specified directory.
    """
    tree_files = []
    parameter_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".nwk"):
                # Load tree file
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    tree_str = f.read().strip()
                    tree_files.append(tree_str)

                # Load corresponding parameter file
                param_file = os.path.splitext(file)[0] + ".params"
                param_path = os.path.join(root, param_file)
                parameter_files.append(param_path)

    return tree_files, parameter_files

# Preprocess data: encode tree structure and labels into numerical format for training
def preprocess_data(tree_files, parameter_files):
    """
    Preprocess data: encode tree structure and labels into numerical format for training.
    """
    # Encode trees and determine the maximum length
    encoded_trees = [encode_tree(tree) for tree in tree_files]
    max_length = max(len(tree) for tree in encoded_trees)
    
    # Pad or truncate the encoded trees to the maximum length
    X = [[node + [0] * (max_length - len(node)) for node in tree] for tree in encoded_trees]
    X = torch.tensor(X, dtype=torch.float32)
    
    # Extract BiSSE parameters
    y = []
    for param_file in parameter_files:
        with open(param_file, "r") as f:
            param_lines = f.readlines()
            param_dict = {}
            for line in param_lines:
                key, value = line.strip().split("=")
                param_dict[key.strip()] = float(value.strip())
            y.append([param_dict["lambda1"], param_dict["lambda2"], param_dict["mu1"], param_dict["mu2"], param_dict["q12"], param_dict["q21"]])

    y = torch.tensor(y, dtype=torch.float32)

    return X, y

# Split data into training and testing sets
def split_data(X, y, test_size=0.2):
    """
    Split data into training and testing sets.
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

# Load data from the specified directory
trees_directory = "trees/bisse"

# Load trees and parameters
trees, parameters = load_data_from_directory(trees_directory)

# Preprocess data
X, y = preprocess_data(trees, parameters)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = split_data(X, y)



TypeError: can only concatenate str (not "list") to str